In [45]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.interpolate import SmoothBivariateSpline
os.chdir(r'C:\Users\Pierre\Desktop\Postdoc\Technical_stuff\PPMS\PPMS_Heat_capacity\PPMS_Heat-capacity_TempCal\He3_TempCal')

In [46]:
calname = '2021-01-15_He3Puck698_multicurve_all_currents.dat'
# Read MFD data from csv file:
# Use last string in header as column header
# Read data from column mfd_idx+2 since, the 3 first columns are x, y, z data
# squeeze=True in order to make it a Series instead of a DataFrame, since there is only one column for each mfd_idx
cal = pd.read_csv(calname, skiprows=7, header=0)
cal

,Temperature (K),Current (code),Active Current Code,Field (Oe),ThRes (ohms),slope of -R vs T (ohms/K),slope of -R vs T (%/K),Point-to-Point R change (ohms),Point-to-Point slope change (ohms/K),Delta R in Overlap (ohms),Delta R in Overlap (%)
0,0.329838,11,11,0.000,9456.026900,63579.680250,672.372032,NaN,NaN,NaN,NaN
1,0.330498,11,11,0.000,9414.240139,63109.659495,670.363817,-41.786761,470.020754,NaN,NaN
2,0.331159,11,11,0.000,9372.679264,62643.327129,668.360939,-41.560874,466.332367,NaN,NaN
3,0.331821,11,11,0.000,9331.342916,62180.652602,666.363386,-41.336349,462.674526,NaN,NaN
4,0.332485,11,11,0.000,9290.229739,61721.605636,664.371144,-41.113177,459.046967,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
42589,346.646201,3,3,8000.156,45.914963,0.082464,0.179602,-0.057158,0.000290,NaN,NaN
42590,347.339494,3,3,8000.156,45.857891,0.082175,0.179196,-0.057072,0.000289,NaN,NaN
42591,348.034172,3,3,8000.156,45.800905,0.081888,0.178791,-0.056986,0.000288,NaN,NaN
42592,348.730241,3,3,8000.156,45.744006,0.081601,0.178387,-0.056900,0.000287,NaN,NaN


In [47]:
cal = cal.iloc[:,:5]
# del cal['Active Current Code']
for cname in cal.columns:
    cal.rename(columns={cname:cname.strip(' ')}, inplace=True)
cal.sort_values('Current (code)', inplace=True, ignore_index=True)
cal

,Temperature (K),Current (code),Active Current Code,Field (Oe),ThRes (ohms)
0,349.427701,3,3,8000.156,45.687192
1,128.162545,3,3,6000.113,86.344235
2,128.418870,3,3,6000.113,86.239695
3,128.675708,3,3,6000.113,86.135242
4,128.933059,3,3,6000.113,86.030876
...,...,...,...,...,...
42589,0.376103,11,11,6000.113,5777.364568
42590,0.376855,11,11,6000.113,5757.329686
42591,0.377609,11,11,6000.113,5737.385169
42592,0.371621,11,11,6000.113,5899.495694


In [48]:
codename = '../2021-01-19_DSPCurrentCodes.txt'
Icodes = pd.read_csv(codename, sep=' ', names=['code', 'current', 'unit'])
Icodes['I (uA)'] = np.where(Icodes['unit']=='nA', Icodes['current']*1e-3,
                            np.where(Icodes['unit']=='mA', Icodes['current']*1e3, 
                                     Icodes['current']))
Icodes.sort_values('code', ignore_index=True)

,code,current,unit,I (uA)
0,0,0.455,mA,455.00
1,1,0.227,mA,227.00
2,2,90.910,uA,90.91
3,3,45.450,uA,45.45
4,4,22.730,uA,22.73
5,5,9.090,uA,9.09
6,6,4.550,uA,4.55
7,7,2.270,uA,2.27
8,8,910.000,nA,0.91
9,9,450.000,nA,0.45


In [49]:
dcal = {}
for code in np.unique(cal['Current (code)']):
    dcal[code] = np.where(cal['Current (code)']==code, Icodes['I (uA)'][Icodes['code']==code].item(), 0)
    
cal['I (uA)'] = np.concatenate([dcal[i][dcal[i]>0] for i in np.unique(cal['Current (code)'])])

In [50]:
cal[cal['Current (code)']==11]

,Temperature (K),Current (code),Active Current Code,Field (Oe),ThRes (ohms),I (uA)
36167,0.383160,11,11,0.000,6866.562161,0.09
36168,0.383927,11,11,0.000,6838.457137,0.09
36169,0.384694,11,11,0.000,6810.497327,0.09
36170,0.374083,11,11,0.000,7215.419410,0.09
36171,0.386235,11,11,0.000,6755.010433,0.09
...,...,...,...,...,...,...
42589,0.376103,11,11,6000.113,5777.364568,0.09
42590,0.376855,11,11,6000.113,5757.329686,0.09
42591,0.377609,11,11,6000.113,5737.385169,0.09
42592,0.371621,11,11,6000.113,5899.495694,0.09


In [80]:
maxT = 2
subcal = cal[cal['Temperature (K)']<maxT]
xi = subcal['I (uA)']# current
xm = subcal['Field (Oe)']# magnetic field
z = subcal['Temperature (K)']# temperature
xr = subcal['ThRes (ohms)']# resistance
Xi = np.unique(xi)
Xm = np.arange(0, 1.1e4, 1e2)#
# Xt = np.arange(min(y), maxT, .01)
Xr = np.arange(7e2, 1.0e4, 1e2)#np.unique(np.round(xr,-1))
# XI, XM, XT = np.meshgrid(Xi, Xm, Xt)  # 3D grid for interpolation
X, Y = np.meshgrid(Xm, Xr)# grid for interpolation
# interp = LinearNDInterpolator(list(zip(xi, xm, xt)), z)
# interp = LinearNDInterpolator(list(zip(xm, xr)), z)
# Z = interp(XI, XM, XT)
# Z = interp(X, Y)

In [ ]:
spline = SmoothBivariateSpline(xm, xr, z, kx=1, ky=1)
Zspline_full = spline(Xm, Xr)
spline_sel = Zspline_full.T<subcal['Temperature (K)'].min()
Zs = Zspline_full.T
dspline = {}
for axis in ['X', 'Y', 'Zs']:
    dspline[axis] = np.ma.masked_where(spline_sel, eval(axis))

In [ ]:
plt.figure()
# plt.pcolormesh(X, Y, Z, shading='auto')
plt.pcolor(dspline['X'], dspline['Y'], dspline['Zs'], shading='auto', vmin=0.)
plt.set_cmap('viridis')
cb = plt.colorbar()
plt.scatter(xm, xr, c=z)
plt.gray()
plt.xlabel(xm.name)
plt.ylabel(xr.name)
cb.set_label(z.name)
plt.xlim(right=1e4)
# plt.legend()
plt.show()